<a href="https://colab.research.google.com/github/robgon-art/DeepLimericks/blob/main/Deep_Limericks_Interactive_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Limericks**
**I once trained an AI to rhyme, It took GPT-J a long time,</br>
Since the Colab was slow, I upgraded to Pro,</br> Each limerick cost me a dime.**

By Robert. A Gonsalves</br>

![image](https://raw.githubusercontent.com/robgon-art/DeepLimericks/main/deep_limericks_med.jpg)

You can see my article on Medium.

The source code and generated Haikus are released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/).</br>
![CC BYC-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)


In [ ]:
#@title Initiialize System

!nvidia-smi -L
!pip install transformers==4.22.2
!pip install bitsandbytes==0.34.0

import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from tqdm.auto import tqdm

class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = torch.clone(DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias))
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

!gdown 1-slQ8AZIPzzIiF9XUhEGwiuantLQ1caz

gpt = torch.load("/content/deep_limericks.pt",  map_location=torch.device('cuda'))
torch.cuda.empty_cache()

In [5]:
#@title Enter a topic for your limerick
#@markdown Leave blank to use randomly generated topics.
topic = 'kitten' #@param {type:"string"}
has_topic = len(topic) > 0
if has_topic:
  print("topic:", topic)
print()

with torch.no_grad():
  prompt1 = "<" + topic + " =T2R="
  result_length = 30
  inputs = tokenizer(prompt1, return_tensors="pt").to('cuda:0')
  beam_outputs = gpt.generate(inputs["input_ids"],
    max_length=result_length,
    top_k=50, top_p=0.95, 
    do_sample=True, temperature=0.7, pad_token_id=50256,
    num_return_sequences=10)
  
  rhyme_sets = []
  topics = []

  for beam_output in beam_outputs:
    text = tokenizer.decode(beam_output, skip_special_tokens=True)
    rhyme_set = text[text.find(" =T2R= ")+len(" =T2R= "):text.rfind(">")]
    parts = rhyme_set.split(" \ ")
    rhymes = []
    for p in parts:
      rhyme = p.strip()
      if len(rhyme) > 0 and rhyme not in rhymes:
        rhymes.append(rhyme)
    if rhyme_set not in rhyme_sets and len(rhymes) == 5:
      rhyme_sets.append(rhyme_set)
      if not has_topic:
        topics.append(rhymes[0])
torch.cuda.empty_cache()

# rhyme_sets.sort()

print("rhyme sets:\n" + 85  * '-')
for i, r in enumerate(rhyme_sets):
  d = r.replace("\\", "/")
  if has_topic:
    print(str(i+1) +":", d)
  else:
    print(str(i+1) +":", "topic:", topics[i]+":", d)

topic: kitten

rhyme sets:
-------------------------------------------------------------------------------------
1: cat / that / meek / speak / flat
2: cat / flat / purr / her / that
3: cat / flat / there / air / that
4: say / today / kitty / pretty / away
5: britain / kitten / hair / there / smitten
6: cat / rat / found / ground / that


In [14]:
#@title Choose a rhyme-set

import re
rhyme_set_selection = 5 #@param {type:"slider", min:1, max:10, step:1}
rss = min(rhyme_set_selection, len(rhyme_sets))
rhyme_text = rhyme_sets[rss-1]

if not has_topic:
  topic = topics[rss-1]
print("topic:", topic)
print("rhyme-set:", rhyme_text.replace("\\", "/"))
print()

with torch.no_grad():
  prompt2 = "<" + topic + ": " + rhyme_text + " =R2L= "
  result_length = 40
  inputs = tokenizer(prompt2, return_tensors="pt").to('cuda:0')
  beam_outputs = gpt.generate(inputs["input_ids"],
    max_length=result_length,
    top_k=50, top_p=0.95, 
    do_sample=True, temperature=0.7, pad_token_id=50256,
    num_return_sequences=10)

  first_rhyme = rhyme_text.split(" \ ")[0].strip().lower()
  first_lines = []

  for beam_output in beam_outputs:
    text = tokenizer.decode(beam_output, skip_special_tokens=True)
    text = ' '.join(text.split())
    trimmed_text = text[text.find(" =R2L= ")+len(" =R2L= "):text.rfind(">")]
    lines = trimmed_text.split(" / ")
    if len(lines) >=  1  and len(lines[0]) > 0:
      first_line = lines[0]
      last_word = first_line.split()[-1].strip().lower()
      last_word = re.sub(r'[^a-z]+', '', last_word)
      if first_rhyme == last_word and first_line not in first_lines:
        first_lines.append(first_line)
torch.cuda.empty_cache()

first_lines.sort()
print("first lines:\n" + 85  * '-')
for i, f in enumerate(first_lines):
  print(str(i+1) +":", f)

topic: kitten
rhyme-set: britain / kitten / hair / there / smitten

first lines:
-------------------------------------------------------------------------------------
1: "Aye, aye, sir!" cried Harry, Britain,
2: "It's an animal (that's Britain),
3: "You are not from Britain?"
4: A young lady named Daisy from Britain
5: I've had some bad times in Britain
6: It's my kitten, the one from Britain
7: The cat's meow was heard in Britain
8: The dandies of Britain were Britain,


In [19]:
#@title Choose the first line
first_line_selection = 4 #@param {type:"slider", min:1, max:10, step:1}
fls = min(first_line_selection, len(first_lines))
first_line = first_lines[fls-1]

print("topic:", topic)
print("rhyme-set:", rhyme_text.replace("\\", "/"))
print()
print("first_line:", first_line)
print()

with torch.no_grad():
  prompt3 = "<" + topic + ": " + rhyme_text + " =R2L= " + first_line + " / "
  result_length = 80
  inputs = tokenizer(prompt3, return_tensors="pt").to('cuda:0')
  beam_outputs = gpt.generate(inputs["input_ids"],
    max_length=result_length,
    top_k=50, top_p=0.95, 
    do_sample=True, temperature=0.7, pad_token_id=50256,
    num_return_sequences=10)

  second_rhyme = rhyme_text.split(" \ ")[1].strip().lower()
  second_lines = []

  for beam_output in beam_outputs:
    text = tokenizer.decode(beam_output, skip_special_tokens=True)
    text = ' '.join(text.split())
    trimmed_text = text[text.find(" =R2L= ")+len(" =R2L= "):text.rfind(">")]
    lines = trimmed_text.split(" / ")
    if len(lines) >= 2 and len(lines[1]) > 0:
      second_line = lines[1]
      last_word = second_line.split()[-1].strip().lower()
      last_word = re.sub(r'[^a-z]+', '', last_word)
      if second_rhyme == last_word and second_line not in second_lines:
        second_lines.append(second_line)
torch.cuda.empty_cache()

second_lines.sort()
print("second lines:\n" + 85  * '-')
for i, f in enumerate(second_lines):
  print(str(i+1) +":", f)

topic: kitten
rhyme-set: britain / kitten / hair / there / smitten

first_line: A young lady named Daisy from Britain

second lines:
-------------------------------------------------------------------------------------
1: Is a kitten who loves to be kitten.
2: Is a little girl. She's a kitten,
3: Met a fellow called Tommy, a kitten.
4: Sought a man who was somewhat a kitten.
5: Was a very attractive young kitten.
6: Was averse to a cat, and a kitten.
7: Went out for a walk with a kitten.


In [21]:
#@title Choose the second line

import re
second_line_selection = 7 #@param {type:"slider", min:1, max:10, step:1}
sls = min(second_line_selection, len(second_lines))
second_line = second_lines[sls-1]

print("topic:", topic)
print("rhyme-set:", rhyme_text.replace("\\", "/"))
print()
print("first_line: ", first_line)
print("second_line:", second_line)
print()

with torch.no_grad():
  prompt4 = "<" + topic + ": " + rhyme_text + " =R2L= " + first_line 
  prompt4 += " / " + second_line + " / "
  result_length = 60
  inputs = tokenizer(prompt4, return_tensors="pt").to('cuda:0')
  beam_outputs = gpt.generate(inputs["input_ids"],
    max_length=result_length,
    top_k=50, top_p=0.95, 
    do_sample=True, temperature=0.7, pad_token_id=50256,
    num_return_sequences=10)

  third_rhyme = rhyme_text.split(" \ ")[2].strip().lower()
  third_lines = []

  for beam_output in beam_outputs:
    text = tokenizer.decode(beam_output, skip_special_tokens=True)
    text = ' '.join(text.split())
    trimmed_text = text[text.find(" =R2L= ")+len(" =R2L= "):text.rfind(">")]
    lines = trimmed_text.split(" / ")
    if len(lines) >= 3 and len(lines[2]) > 0:
      third_line = lines[2]
      last_word = third_line.split()[-1].strip().lower()
      last_word = re.sub(r'[^a-z]+', '', last_word)
      if third_rhyme == last_word and third_line not in third_lines:
        third_lines.append(third_line)
torch.cuda.empty_cache()

third_lines.sort()
print("third lines:\n" + 85  * '-')
for i, f in enumerate(third_lines):
  print(str(i+1) +":", f)

topic: kitten
rhyme-set: britain / kitten / hair / there / smitten

first_line:  A young lady named Daisy from Britain
second_line: Went out for a walk with a kitten.

third lines:
-------------------------------------------------------------------------------------
1: But the kitten had hair,
2: She removed all her hair,
3: She was wearing her hair
4: She'd just cut her hair, 
5: When she saw her hair,


In [23]:
#@title Choose the third line

import re
third_line_selection = 4 #@param {type:"slider", min:1, max:10, step:1}
tls = min(third_line_selection, len(third_lines))
third_line = third_lines[tls-1]

print("topic:", topic)
print("rhyme-set:", rhyme_text.replace("\\", "/"))
print()
print("first_line :", first_line)
print("second_line:", second_line)
print("third_line :", third_line)
print()

with torch.no_grad():
  prompt5 = "<" + topic + ": " + rhyme_text + " =R2L= " + first_line 
  prompt5 += " / " + second_line + " / " + third_line + " / "
  result_length = 70
  inputs = tokenizer(prompt5, return_tensors="pt").to('cuda:0')
  beam_outputs = gpt.generate(inputs["input_ids"],
    max_length=result_length,
    top_k=50, top_p=0.95, 
    do_sample=True, temperature=0.7, pad_token_id=50256,
    num_return_sequences=10)

  fourth_rhyme = rhyme_text.split(" \ ")[3].strip().lower()
  fourth_lines = []

  for beam_output in beam_outputs:
    text = tokenizer.decode(beam_output, skip_special_tokens=True)
    text = ' '.join(text.split())
    trimmed_text = text[text.find(" =R2L= ")+len(" =R2L= "):text.rfind(">")]
    lines = trimmed_text.split(" / ")
    if len(lines) >= 4 and len(lines[3]) > 0:
      fourth_line = lines[3]
      last_word = fourth_line.split()[-1].strip().lower()
      last_word = re.sub(r'[^a-z]+', '', last_word)
      if fourth_rhyme == last_word and fourth_line not in fourth_lines:
        fourth_lines.append(fourth_line)
torch.cuda.empty_cache()

fourth_lines.sort()
print("fourth lines:\n" + 85  * '-')
for i, f in enumerate(fourth_lines):
  print(str(i+1) +":", f)

topic: kitten
rhyme-set: britain / kitten / hair / there / smitten

first_line : A young lady named Daisy from Britain
second_line: Went out for a walk with a kitten.
third_line : She'd just cut her hair, 

fourth lines:
-------------------------------------------------------------------------------------
1: And her face was there.
2: And she thought that was there,
3: And the kitten was there,
4: And the kitten was there.
5: And was wondering there
6: But her date? He was there!
7: But the kitten was there.
8: When she heard, "You're there!"


In [25]:
#@title Choose the fourth line

import re
fourth_line_selection = 6 #@param {type:"slider", min:1, max:10, step:1}
fls = min(fourth_line_selection, len(fourth_lines))
fourth_line = fourth_lines[fls-1]
print("topic:", topic)
print("rhyme-set:", rhyme_text.replace("\\", "/"))
print()
print("first_line  :", first_line)
print("second_line :", second_line)
print("third_line  :", third_line)
print("fourth_line :", fourth_line)
print()

with torch.no_grad():
  prompt5 = "<" + topic + ": " + rhyme_text + " =R2L= " + first_line 
  prompt5 += " / " + second_line + " / " + third_line + " / "
  prompt5 += fourth_line + " / "
  result_length = 150
  inputs = tokenizer(prompt5, return_tensors="pt").to('cuda:0')
  beam_outputs = gpt.generate(inputs["input_ids"],
    max_length=result_length,
    top_k=50, top_p=0.95, 
    do_sample=True, temperature=0.7, pad_token_id=50256,
    num_return_sequences=10)

  fifth_rhyme = rhyme_text.split(" \ ")[4].strip().lower()
  fifth_lines = []

  for beam_output in beam_outputs:
    text = tokenizer.decode(beam_output, skip_special_tokens=True)
    text = ' '.join(text.split())
    trimmed_text = text[text.find(" =R2L= ")+len(" =R2L= "):text.rfind(">")]
    lines = trimmed_text.split(" / ")
    if len(lines) >= 5 and len(lines[4]) > 0:
      fifth_line = lines[4]
      last_word = fifth_line.split()[-1].strip().lower()
      last_word = re.sub(r'[^a-z]+', '', last_word)
      if fifth_rhyme == last_word and fifth_line not in fifth_lines:
        fifth_lines.append(fifth_line)
torch.cuda.empty_cache()

fifth_lines.sort()
print("fifth lines:\n" + 85  * '-')
for i, f in enumerate(fifth_lines):
  print(str(i+1) +":", f)

topic: kitten
rhyme-set: britain / kitten / hair / there / smitten

first_line  : A young lady named Daisy from Britain
second_line : Went out for a walk with a kitten.
third_line  : She'd just cut her hair, 
fourth_line : But her date? He was there!

fifth lines:
-------------------------------------------------------------------------------------
1: "Are you OK?" asked the lad. "Am I smitten?"
2: He was smitten, but not really smitten.
3: He was smitten. She was not. He was smitten.
4: Now she's mad at herself, she's so smitten!
5: She was caught by her date, and she's smitten.
6: She's too late: she's a-knitting and smitten.


In [26]:
#@title Choose final line

import re
fifth_line_selection = 1 #@param {type:"slider", min:1, max:10, step:1}
vls = min(fifth_line_selection, len(fifth_lines))
fifth_line = fifth_lines[vls-1]
print("Final Limerick")
print()
print(first_line)
print(second_line)
print(third_line)
print(fourth_line)
print(fifth_line)

Final Limerick

A young lady named Daisy from Britain
Went out for a walk with a kitten.
She'd just cut her hair, 
But her date? He was there!
"Are you OK?" asked the lad. "Am I smitten?"
